<a href="https://colab.research.google.com/github/taniokah/Sports_Analysis/blob/master/Background_Subtract_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 手順

1. Google Drive に接続
2. 必要な動画をコピー
3. 必要なライブラリをpip install
4. プログラムの改変
5. プログラムの実行
6. 内容の確認


参考文献: https://www.youtube.com/watch?v=nRt2LPRz704  
https://medium.com/@kananvyas/player-and-football-detection-using-opencv-python-in-fifa-match-6fd2e4e373f0  
https://temari.co.jp/blog/2017/11/13/opencv-4/  

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp /content/drive/'My Drive'/test/'20190224 鹿屋体育大学 vs 鹿児島高専 1st-l00CwQ6HI_o_5m.mp4' /content/'20190224 鹿屋体育大学 vs 鹿児島高専 1st-l00CwQ6HI_o_5m.mp4'

In [3]:
import tensorflow as tf
print(tf.__version__)

!pip install numpy
!pip install scikit-learn
!pip install opencv-python
!pip install opencv-contrib-python
!pip install pillow


2.2.0-rc3


In [9]:
# importing libraries 
import numpy as np
import cv2
import sys
  
# creating object 
fgbg2 = cv2.bgsegm.createBackgroundSubtractorMOG(history=10)
  
# capture frames from a camera  
cap = cv2.VideoCapture('20190224 鹿屋体育大学 vs 鹿児島高専 1st-l00CwQ6HI_o_5m.mp4')

video_FourCC = cv2.VideoWriter_fourcc(*"mp4v")
video_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT)
video_fps = cap.get(cv2.CAP_PROP_FPS)
video_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
out = cv2.VideoWriter('20190224 鹿屋体育大学 vs 鹿児島高専 1st-l00CwQ6HI_o_5m_ball.mp4', video_FourCC, video_fps, video_size)

preview = False
color = True

count = 0
while count < int(video_frame): 
    # read frames 
    ret, img = cap.read()
    if ret == False:
        break
    count += 1
    if count < 0:
        continue
    if count > sys.maxsize:
        break
    print("\r" + str(count) + "/" + str(int(video_frame)), end="", file=sys.stderr)
    
    height = img.shape[0]
    width = img.shape[1]
      
    # apply mask for background subtraction 
    fgmask2 = fgbg2.apply(img)
    
    (contours, hierarchy) = cv2.findContours(fgmask2.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
        area = cv2.contourArea(c)
        #if area < 10:
        #    continue

        #get bounding box from countour
        (x, y, w, h) = cv2.boundingRect(c)
        if x < (width / 4 * 0.2) or x > (width / 4 * 3.8):
            continue
        if y < (height / 8) or y > (height / 2):
            continue
        if y < (height / 4) / (width / 4) * (x - (width / 4 * 3)) or y < (height / 4) / (width / 4) * ((width / 4) - x):
            continue
        if w > 20 or h > 20 or abs(w - h) / w > 0.5:
            continue
            
        #draw bounding box
        if color:
            ball_img = img[y:y+h, x:x+w]
            
            ball_hsv = cv2.cvtColor(ball_img, cv2.COLOR_BGR2HSV_FULL)
            # green
            #lower_green = np.array([75, 50, 50])
            #upper_green = np.array([110, 255, 255])
            # white
            sensitivity = 150
            lower_white = np.array([0,0,50])
            upper_white = np.array([180,255,255])
            mask = cv2.inRange(ball_hsv, lower_white, upper_white)
            res = cv2.bitwise_and(ball_hsv, ball_hsv, mask=mask)
            res = cv2.cvtColor(res,cv2.COLOR_HSV2BGR)
            res = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
            nzCount = cv2.countNonZero(res)
            if nzCount > 0:
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        else:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    out.write(img)

    if preview:
        cv2.namedWindow("Original",cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Original",1280,680)
        #img = cv2.resize(img)
        cv2.imshow('Original', img)
        #cv2.imshow('MOG2', fgmask2)
    
        k = cv2.waitKey(25) & 0xff
        if k == 27: 
            break

out.release()
cap.release()
cv2.destroyAllWindows()

100/9000

In [0]:
!cp '20190224 鹿屋体育大学 vs 鹿児島高専 1st-l00CwQ6HI_o_5m_ball.mp4' drive/My\ Drive/test/"20190224 鹿屋体育大学 vs 鹿児島高専 1st-l00CwQ6HI_o_5m_ball.mp4"